# 문제 2: 치와와 vs 머핀 분류 (5겹 교차검증 × 5회 반복)

## 개요
- **데이터셋**: Kaggle Muffin vs Chihuahua (128×128 RGB)
- **검증 방법**: RepeatedStratifiedKFold (5겹 × 5회 = 25회 학습)
- **평가 척도**: Accuracy, F1 Score (Micro), F1 Score (Macro)

## 검증 방법: RepeatedStratifiedKFold

```
RepeatedStratifiedKFold(n_splits=5, n_repeats=5)

반복 1: Fold1 → Fold2 → Fold3 → Fold4 → Fold5
반복 2: Fold1 → Fold2 → Fold3 → Fold4 → Fold5
반복 3: Fold1 → Fold2 → Fold3 → Fold4 → Fold5
반복 4: Fold1 → Fold2 → Fold3 → Fold4 → Fold5
반복 5: Fold1 → Fold2 → Fold3 → Fold4 → Fold5

총 25회 학습 → 평균 ± 표준편차 산출
```

**장점**:
- 단순 5겹 교차검증보다 더 안정적인 성능 추정
- 서로 다른 무작위 분할로 분산 감소
- sklearn에서 공식 제공하는 반복 교차검증 방법

## 문제의 특성
치와와와 머핀은 외형이 매우 유사합니다:
- **둥근 형태**: 치와와의 얼굴과 머핀의 윗면 모두 원형
- **갈색 톤**: 치와와의 털색과 머핀의 반죽색이 유사
- **점 패턴**: 치와와의 눈/코와 머핀의 초콜릿칩이 유사

→ 세밀한 특징 추출이 필요한 도전적인 분류 문제

## 구현 모델 (6개)

### 베이스라인 모델
1. **LeNet-5** (LeCun, 1998) - 최초의 성공적 CNN, 수업시간에 배운 모델

### 고급 모델
2. **VGGNet-style** (Simonyan, 2014) - 작은 3×3 커널을 깊게 쌓은 구조
3. **ResNetCNN** (He, 2015) - Residual Connection으로 깊은 학습
4. **EfficientNet-Lite** - 간소화된 MBConv + SE 구조
5. **EfficientNet-B0** (Tan, 2019) - 원본 B0 아키텍처
6. **ConvNeXt-Tiny** (Liu, 2022) - ViT의 설계 원칙을 CNN에 적용

## 평가 척도 설명
- **Accuracy**: 전체 정확도 (TP + TN) / Total
- **F1 (Micro)**: 전체 TP/FP/FN 합산 후 계산 → 균형 데이터에서 Accuracy와 동일
- **F1 (Macro)**: 클래스별 F1의 평균 → 불균형 데이터에서 소수 클래스 성능 반영

## 1. 라이브러리

In [ ]:
# -------------------------------
# 공용 라이브러리 및 유틸 불러오기
# - torch/torchvision: 모델 정의, 변환, Mixed Precision
# - sklearn: 교차검증 및 지표 계산
# - PIL: 이미지 로딩
# - matplotlib: 시각화 및 한글 폰트 설정
# - zipfile/os: Kaggle 데이터 압축 해제 및 경로 관리
# -------------------------------
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.amp import autocast, GradScaler  # Mixed Precision (새 API)
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import RepeatedStratifiedKFold  # 5겹 × 5회 반복
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os
import zipfile

# 한글 폰트 설정 (Linux 서버 환경) - 그래프 라벨 깨짐 방지
def set_korean_font():
    """시스템에서 사용 가능한 한글 폰트를 자동 설정"""
    font_candidates = [
        'NanumGothic', 'NanumBarunGothic', 'Malgun Gothic',
        'AppleGothic', 'DejaVu Sans', 'Noto Sans CJK KR'
    ]
    available_fonts = [f.name for f in fm.fontManager.ttflist]
    
    for font in font_candidates:
        if font in available_fonts:
            plt.rcParams['font.family'] = font
            plt.rcParams['axes.unicode_minus'] = False
            print(f"한글 폰트 설정: {font}")
            return
    
    print("한글 폰트를 찾지 못했습니다. 영문 라벨을 사용합니다.")

set_korean_font()

# GPU 우선, Mac이면 MPS, 그 외 CPU 선택
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Device: {device}")

# GPU 정보 출력 (리포트용)
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# --------------------------------------------
# H100 GPU 최적화 기본 설정 + 시드 고정 헬퍼
# --------------------------------------------
BATCH_SIZE = 128          # H100: 80GB 메모리, 배치 크기 증가
NUM_WORKERS = 8           # 데이터 로딩 병렬화
PIN_MEMORY = True         # GPU 메모리 전송 최적화
USE_AMP = True            # Mixed Precision (BF16/FP16)

def set_seed(seed):
    """torch/np 시드를 모두 고정하여 StratifiedKFold 일관성 유지"""
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

# cuDNN 최적화 (고정 입력 크기에서 성능 향상)
torch.backends.cudnn.benchmark = True

print(f"H100 최적화 설정:")
print(f"  - Batch Size: {BATCH_SIZE}")
print(f"  - Num Workers: {NUM_WORKERS}")
print(f"  - Pin Memory: {PIN_MEMORY}")
print(f"  - Mixed Precision (AMP): {USE_AMP}")
print(f"  - cuDNN Benchmark: {torch.backends.cudnn.benchmark}")



## 2. 데이터셋 다운로드

Kaggle API 필요: `pip install kaggle`

In [ ]:
# --------------------------------------------
# Kaggle 데이터 다운로드 디렉토리 준비
# --------------------------------------------
DATA_DIR = './data/chihuahua_muffin'
os.makedirs(DATA_DIR, exist_ok=True)



In [ ]:
# --------------------------------------------
# Kaggle 데이터셋 다운로드 (치와와 vs 머핀)
# --------------------------------------------
!kaggle datasets download -d samuelcortinhas/muffin-vs-chihuahua-image-classification -p {DATA_DIR}



In [ ]:
# --------------------------------------------
# 다운로드한 zip 압축 해제
# --------------------------------------------
zip_path = os.path.join(DATA_DIR, 'muffin-vs-chihuahua-image-classification.zip')
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(DATA_DIR)
print("압축 해제 완료")



## 3. 데이터 준비

In [ ]:
# --------------------------------------------
# Custom Dataset 정의
# - 폴더 구조: chihuahua/, muffin/
# - 이미지 경로와 라벨을 미리 스캔하여 리스트로 보관
# --------------------------------------------
class ChihuahuaMuffinDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        for class_name, class_idx in [('chihuahua', 0), ('muffin', 1)]:
            class_dir = os.path.join(data_dir, class_name)
            if os.path.exists(class_dir):
                for img_name in os.listdir(class_dir):
                    if img_name.endswith(('.jpg', '.jpeg', '.png')):
                        self.image_paths.append(os.path.join(class_dir, img_name))
                        self.labels.append(class_idx)
        
        self.labels = np.array(self.labels)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # PIL로 로드 후 RGB 변환, transform 적용
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]



In [ ]:
# --------------------------------------------
# 데이터 변환 정의
# - train: 색상/밝기 증강 포함 (클래스 유사도 대응)
# - test: 증강 없이 정규화만 적용
# --------------------------------------------
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])



In [ ]:
# --------------------------------------------
# 데이터셋 로드 (train/test 분리 제공)
# --------------------------------------------
train_dir = os.path.join(DATA_DIR, 'train')
test_dir = os.path.join(DATA_DIR, 'test')

train_dataset = ChihuahuaMuffinDataset(train_dir)
test_dataset = ChihuahuaMuffinDataset(test_dir, transform=test_transform)

print(f"훈련: {len(train_dataset)}, 테스트: {len(test_dataset)}")
print(f"클래스 분포: chihuahua={sum(train_dataset.labels==0)}, muffin={sum(train_dataset.labels==1)}")



In [ ]:
# --------------------------------------------
# 샘플 시각화 - 치와와 vs 머핀의 유사성 확인
# --------------------------------------------
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
temp_ds = ChihuahuaMuffinDataset(train_dir, transform=test_transform)

chi_idx = np.where(temp_ds.labels == 0)[0][:5]
muf_idx = np.where(temp_ds.labels == 1)[0][:5]

for i, idx in enumerate(chi_idx):
    img, _ = temp_ds[idx]
    img = img * torch.tensor([0.229, 0.224, 0.225]).view(3,1,1) + torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
    axes[0, i].imshow(img.permute(1,2,0).clip(0,1))
    axes[0, i].set_title('Chihuahua')
    axes[0, i].axis('off')

for i, idx in enumerate(muf_idx):
    img, _ = temp_ds[idx]
    img = img * torch.tensor([0.229, 0.224, 0.225]).view(3,1,1) + torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
    axes[1, i].imshow(img.permute(1,2,0).clip(0,1))
    axes[1, i].set_title('Muffin')
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()



## 4. 모델 정의

### 망 구조 표기법
- **C**: Convolution (합성곱) - 공간적 특징 추출
- **DWC**: Depthwise Separable Conv - 채널별 독립 합성곱
- **BN**: Batch Normalization - 학습 안정화
- **R**: ReLU / **T**: Tanh / **Swish**: x·σ(x) - 활성화 함수
- **AvgP** / **MaxP**: Average/Max Pooling
- **GAP**: Global Average Pooling
- **D**: Dropout - 정규화
- **FC**: Fully Connected
- **Res**: Residual Connection
- **SE**: Squeeze-and-Excitation - 채널 attention
- **MBConv**: Mobile Inverted Bottleneck Conv

---

### 4.1 LeNet-5 (베이스라인, LeCun et al., 1998)

**구조**: `C(3,6,5×5) → T → AvgP → C(6,16,5×5) → T → AvgP → C(16,32,5×5) → T → AvgP → Flat → FC → T → FC → T → FC`

**특징**:
- 최초의 성공적인 CNN (수업시간에 배운 모델)
- 5×5 큰 커널로 특징 추출
- Tanh 활성화 함수 사용

**효과/한계**:
- ✅ CNN의 기본 원리 적용
- ❌ 얕은 구조로 세밀한 특징 추출 한계
- ❌ 치와와 눈과 머핀 초콜릿칩 구분 어려움

In [ ]:
class LeNet5(nn.Module):
    """
    LeNet-5 (LeCun et al., 1998) - 128x128 RGB 버전
    구조: C(5x5) → T → AvgP → C(5x5) → T → AvgP → C(5x5) → T → AvgP → Flat → FC → T → FC → T → FC
    """
    def __init__(self, num_classes=2):
        super().__init__()
        # 레이어를 ModuleList에 저장해 구조를 한눈에 파악
        self.layers = nn.ModuleList([
            nn.Conv2d(3, 6, 5),                # C(3,6,5x5): 128 → 124
            nn.Tanh(),                          # T
            nn.AvgPool2d(2, 2),                 # AvgP: 124 → 62
            nn.Conv2d(6, 16, 5),                # C(6,16,5x5): 62 → 58
            nn.Tanh(),                          # T
            nn.AvgPool2d(2, 2),                 # AvgP: 58 → 29
            nn.Conv2d(16, 32, 5),               # C(16,32,5x5): 29 → 25
            nn.Tanh(),                          # T
            nn.AvgPool2d(2, 2),                 # AvgP: 25 → 12
            nn.Flatten(),                       # Flat: (B,32,12,12) → (B,4608)
            nn.Linear(32 * 12 * 12, 120),       # FC
            nn.Tanh(),                          # T
            nn.Linear(120, 84),                 # FC
            nn.Tanh(),                          # T
            nn.Linear(84, num_classes)          # FC: 출력
        ])
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

print("LeNet-5: C(5x5) → T → AvgP → C(5x5) → T → AvgP → C(5x5) → T → AvgP → Flat → FC×3")
print(LeNet5())
print(f"파라미터: {sum(p.numel() for p in LeNet5().parameters()):,}")



### 4.2 VGGNet-style (Simonyan & Zisserman, 2014)

**구조**: `[C(3×3) → BN → R]×2 → MaxP → [C(3×3) → BN → R]×2 → MaxP → [C(3×3) → BN → R]×3 → MaxP → [C(3×3) → BN → R]×3 → GAP → D → FC`

**특징**:
- 작은 3×3 커널을 깊게 쌓음 (VGG의 핵심)
- 3×3 두 번 = 5×5 수용 영역, 더 적은 파라미터
- BN + ReLU로 현대화

**효과**:
- ✅ 깊은 네트워크로 세밀한 패턴 학습
- ✅ 작은 커널로 파라미터 효율성
- ⚠️ Skip connection 없어 gradient 문제 가능

In [ ]:
class VGGBlock(nn.Module):
    """VGG Block: [C(3x3) → BN → R] × n_convs"""
    def __init__(self, in_ch, out_ch, n_convs=2):
        super().__init__()
        layers = []
        for i in range(n_convs):
            layers.extend([
                nn.Conv2d(in_ch if i == 0 else out_ch, out_ch, 3, 1, 1),
                nn.BatchNorm2d(out_ch),
                nn.ReLU()
            ])
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


class VGGNet(nn.Module):
    """
    VGGNet-style - 128x128 RGB
    구조: VGGBlock×4 + MaxPool×4 + GAP + FC
    """
    def __init__(self, num_classes=2):
        super().__init__()
        self.layers = nn.ModuleList([
            VGGBlock(3, 32, n_convs=2),         # Block1: 128→64
            nn.MaxPool2d(2, 2),
            VGGBlock(32, 64, n_convs=2),        # Block2: 64→32
            nn.MaxPool2d(2, 2),
            VGGBlock(64, 128, n_convs=3),       # Block3: 32→16
            nn.MaxPool2d(2, 2),
            VGGBlock(128, 256, n_convs=3),      # Block4: 16→8
            nn.MaxPool2d(2, 2),
            nn.AdaptiveAvgPool2d(1),            # GAP
            nn.Dropout(0.5)
        ])
        self.fc = nn.Linear(256, num_classes)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

print("VGGNet: [C(3x3)→BN→R]×n → MaxP (4 blocks) → GAP → D → FC")
print(VGGNet())
print(f"파라미터: {sum(p.numel() for p in VGGNet().parameters()):,}")



### 4.3 ResNetCNN (He et al., 2015)

**구조**: `C(3,32,7×7,s2) → BN → R → MaxP → Res(32,64) → Res(64,128) → Res(128,256) → GAP → D(0.5) → FC(256,2)`

**ResBlock 내부**: `C(3×3) → BN → R → C(3×3) → BN → (+shortcut) → R`

**특징**:
- Residual Connection (skip connection)
- H(x) = F(x) + x로 잔차 학습

**효과**:
- ✅ Skip connection으로 gradient flow 개선
- ✅ Gradient vanishing 문제 해결
- ✅ VGGNet보다 깊게 학습 가능

In [ ]:
class ResidualBlock(nn.Module):
    """Residual Block: C → BN → R → C → BN → (+shortcut) → R"""
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Conv2d(in_ch, out_ch, 3, stride, 1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(),
            nn.Conv2d(out_ch, out_ch, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_ch)
        ])
        self.shortcut = nn.Sequential()
        if stride != 1 or in_ch != out_ch:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 1, stride, bias=False),
                nn.BatchNorm2d(out_ch)
            )
    
    def forward(self, x):
        out = x
        for layer in self.layers:
            out = layer(out)
        out += self.shortcut(x)
        return nn.ReLU()(out)


class ResNetCNN(nn.Module):
    """
    ResNetCNN - 128x128 RGB
    구조: C(7x7) → BN → R → MaxP → Res×3 → GAP → D → FC
    """
    def __init__(self, num_classes=2):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Conv2d(3, 32, 7, 2, 3, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(3, 2, 1),
            ResidualBlock(32, 64, 1),
            ResidualBlock(64, 128, 2),
            ResidualBlock(128, 256, 2),
            nn.AdaptiveAvgPool2d(1),
            nn.Dropout(0.5)
        ])
        self.fc = nn.Linear(256, num_classes)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

print("ResNetCNN: C(7x7) → BN → R → MaxP → Res×3 → GAP → D → FC")
print(ResNetCNN())
print(f"파라미터: {sum(p.numel() for p in ResNetCNN().parameters()):,}")



### 4.4 EfficientNet-Lite (간소화된 EfficientNet)

**구조**: `C(3,32,3×3,s2) → BN → Swish → [MBConv]×8 → GAP → D → FC`

**MBConv (Mobile Inverted Bottleneck Conv)**:
`C(1×1,expand) → BN → Swish → DWC(3×3) → BN → Swish → SE → C(1×1,project) → BN → (+shortcut)`

**특징**:
- EfficientNet의 핵심 구성요소만 사용한 간소화 버전
- MBConv + SE + Swish 조합
- 128×128 입력에 맞게 채널 수 축소

**효과**:
- ✅ 파라미터 효율적 (원본 B0보다 가벼움)
- ✅ 빠른 학습 가능
- ⚠️ 원본 B0 대비 표현력 제한

In [ ]:
class Swish(nn.Module):
    """Swish 활성화: x * sigmoid(x), ReLU보다 smooth"""
    def forward(self, x):
        return x * torch.sigmoid(x)


class SEBlock(nn.Module):
    """Squeeze-and-Excitation Block - 채널 attention"""
    def __init__(self, channels, reduction=4):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(channels, channels // reduction),
            Swish(),
            nn.Linear(channels // reduction, channels),
            nn.Sigmoid()
        ])
    
    def forward(self, x):
        b, c, _, _ = x.size()
        scale = x
        for layer in self.layers:
            scale = layer(scale)
        return x * scale.view(b, c, 1, 1)


class MBConv(nn.Module):
    """
    Mobile Inverted Bottleneck Conv (EfficientNet의 핵심)
    구조: C(1x1,expand) → BN → Swish → DWC(kxk) → BN → Swish → SE → C(1x1,project) → BN → (+shortcut)
    
    Args:
        in_ch: 입력 채널
        out_ch: 출력 채널
        kernel_size: Depthwise Conv 커널 크기 (3 or 5)
        expand_ratio: 채널 확장 비율
        stride: Depthwise Conv stride
        use_se: SE Block 사용 여부
    """
    def __init__(self, in_ch, out_ch, kernel_size=3, expand_ratio=4, stride=1, use_se=True):
        super().__init__()
        hidden_ch = in_ch * expand_ratio
        self.use_residual = (stride == 1 and in_ch == out_ch)
        padding = (kernel_size - 1) // 2
        
        layers = []
        # Expand 단계: 채널 확장
        if expand_ratio != 1:
            layers.extend([
                nn.Conv2d(in_ch, hidden_ch, 1, bias=False),
                nn.BatchNorm2d(hidden_ch),
                Swish()
            ])
        else:
            hidden_ch = in_ch
        
        # Depthwise Conv: 채널별 독립 필터
        layers.extend([
            nn.Conv2d(hidden_ch, hidden_ch, kernel_size, stride, padding, groups=hidden_ch, bias=False),
            nn.BatchNorm2d(hidden_ch),
            Swish()
        ])
        
        # SE Block (선택)
        if use_se:
            layers.append(SEBlock(hidden_ch))
        
        # Project 단계: 출력 채널로 축소
        layers.extend([
            nn.Conv2d(hidden_ch, out_ch, 1, bias=False),
            nn.BatchNorm2d(out_ch)
        ])
        
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        out = x
        for layer in self.layers:
            out = layer(out)
        if self.use_residual:
            out = out + x
        return out


class EfficientNetLite(nn.Module):
    """
    EfficientNet-Lite - 간소화된 EfficientNet (128x128 RGB)
    구조: C(3x3,s2) → BN → Swish → MBConv×8 → GAP → D → FC
    """
    def __init__(self, num_classes=2):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Conv2d(3, 32, 3, 2, 1, bias=False),
            nn.BatchNorm2d(32),
            Swish(),
            MBConv(32, 32, kernel_size=3, expand_ratio=1, stride=1),
            MBConv(32, 48, kernel_size=3, expand_ratio=4, stride=2),
            MBConv(48, 48, kernel_size=3, expand_ratio=4, stride=1),
            MBConv(48, 96, kernel_size=3, expand_ratio=4, stride=2),
            MBConv(96, 96, kernel_size=3, expand_ratio=4, stride=1),
            MBConv(96, 192, kernel_size=3, expand_ratio=4, stride=2),
            MBConv(192, 192, kernel_size=3, expand_ratio=4, stride=1),
            MBConv(192, 256, kernel_size=3, expand_ratio=4, stride=1),
            nn.AdaptiveAvgPool2d(1),
            nn.Dropout(0.3)
        ])
        self.fc = nn.Linear(256, num_classes)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

print("EfficientNet-Lite: C(3x3,s2) → BN → Swish → MBConv×8 → GAP → D → FC")
print(f"파라미터: {sum(p.numel() for p in EfficientNetLite().parameters()):,}")



### 4.5 EfficientNet-B0 (Tan & Le, 2019) - 최고 성능 예상

**원본 논문의 B0 아키텍처** (입력: 224×224, 여기서는 128×128 적용)

| Stage | Operator | Resolution | Channels | Layers |
|-------|----------|------------|----------|--------|
| 1 | MBConv1, k3×3 | 112×112 | 16 | 1 |
| 2 | MBConv6, k3×3 | 56×56 | 24 | 2 |
| 3 | MBConv6, k5×5 | 28×28 | 40 | 2 |
| 4 | MBConv6, k3×3 | 14×14 | 80 | 3 |
| 5 | MBConv6, k5×5 | 14×14 | 112 | 3 |
| 6 | MBConv6, k5×5 | 7×7 | 192 | 4 |
| 7 | MBConv6, k3×3 | 7×7 | 320 | 1 |

**구조 특징**:
- **MBConv1**: expand_ratio=1 (채널 확장 없음)
- **MBConv6**: expand_ratio=6 (채널 6배 확장 후 축소)
- **k3×3 / k5×5**: Depthwise Conv 커널 크기
- **Compound Scaling**: 깊이, 너비, 해상도 균형있게 스케일링

**효과**:
- ✅ 원본 논문의 검증된 아키텍처
- ✅ 5×5 커널로 더 넓은 수용 영역
- ✅ 단계적 채널 증가로 세밀한 특징 학습
- ✅ ImageNet에서 검증된 구조

In [ ]:
class EfficientNetB0(nn.Module):
    """
    EfficientNet-B0 (Tan & Le, 2019) - 원본 논문 아키텍처

    입력: 128×128 RGB (원본은 224×224)
    """
    def __init__(self, num_classes=2):
        super().__init__()
        
        # B0 Configuration: (expand_ratio, channels, layers, stride, kernel_size)
        b0_config = [
            (1, 16, 1, 1, 3),
            (6, 24, 2, 2, 3),
            (6, 40, 2, 2, 5),
            (6, 80, 3, 2, 3),
            (6, 112, 3, 1, 5),
            (6, 192, 4, 2, 5),
            (6, 320, 1, 1, 3),
        ]
        
        layers = []
        # Stem: Conv3×3, stride 2, 32ch -> 128→64
        layers.extend([
            nn.Conv2d(3, 32, 3, 2, 1, bias=False),
            nn.BatchNorm2d(32),
            Swish()
        ])
        
        in_ch = 32
        for expand_ratio, out_ch, n_layers, stride, kernel_size in b0_config:
            for i in range(n_layers):
                s = stride if i == 0 else 1
                layers.append(
                    MBConv(in_ch, out_ch, kernel_size=kernel_size, 
                           expand_ratio=expand_ratio, stride=s, use_se=True)
                )
                in_ch = out_ch
        
        layers.extend([
            nn.Conv2d(320, 1280, 1, bias=False),
            nn.BatchNorm2d(1280),
            Swish(),
            nn.AdaptiveAvgPool2d(1),
            nn.Dropout(0.2)
        ])
        
        self.layers = nn.ModuleList(layers)
        self.fc = nn.Linear(1280, num_classes)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

print("EfficientNet-B0: Stem → MBConv×16 → Conv1×1(1280) → GAP → D → FC")
print(f"파라미터: {sum(p.numel() for p in EfficientNetB0().parameters()):,}")



### 4.6 ConvNeXt-Tiny (Liu et al., 2022) - 최신 고성능 CNN

**핵심 아이디어**: Vision Transformer(ViT)의 설계 원칙을 순수 CNN에 적용

**ViT에서 가져온 설계 요소**:
| ViT 특성 | ConvNeXt 적용 |
|----------|--------------|
| 큰 패치 (16×16) | Patchify stem: 4×4 conv, stride 4 |
| LayerNorm | BatchNorm → LayerNorm 교체 |
| 넓은 MLP (4× 확장) | 1×1 conv로 4배 채널 확장 |
| GELU 활성화 | ReLU → GELU 교체 |
| 적은 정규화 | 활성화 함수 1개만 사용 |

**ConvNeXt Block 구조**:
```
DWConv(7×7) → LayerNorm → PWConv(1×1, 4×확장) → GELU → PWConv(1×1, 축소) → LayerScale → + Residual
```

**주요 특징**:
1. **Depthwise Conv 7×7**: 채널별 독립 합성곱으로 넓은 수용 영역
2. **LayerNorm (채널 방향)**: BatchNorm 대신 사용, 배치 크기에 독립적
3. **Inverted Bottleneck**: MobileNet처럼 채널 확장 후 축소
4. **LayerScale**: 잔차 연결 전 학습 가능한 스케일링 (초기값 1e-6)

**효과**:
- ✅ Transformer 수준의 성능을 CNN으로 달성
- ✅ 7×7 Depthwise로 넓은 패턴 인식 (치와와 얼굴 전체 윤곽)
- ✅ LayerScale로 안정적인 깊은 네트워크 학습
- ✅ ImageNet에서 ResNet, Swin Transformer와 경쟁

In [ ]:
# --------------------------------------------
# ConvNeXt-Tiny 스타일 고성능 CNN (128x128 RGB)
# - Depthwise 7x7 → LayerNorm → Pointwise 확장/축소 (GELU)
# - 4-stage ConvNeXt 변형: {3,3,9,3} 블록 + 다운샘플
# --------------------------------------------
class LayerNorm2d(nn.Module):
    """채널 차원 기준 LayerNorm"""
    def __init__(self, num_channels, eps=1e-6):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(num_channels))
        self.bias = nn.Parameter(torch.zeros(num_channels))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(dim=1, keepdim=True)
        var = x.var(dim=1, keepdim=True, unbiased=False)
        x = (x - mean) / torch.sqrt(var + self.eps)
        return self.weight.view(1, -1, 1, 1) * x + self.bias.view(1, -1, 1, 1)


class ConvNeXtBlock(nn.Module):
    """ConvNeXt Block: DWConv(7x7) → LN → PWConv×2 + GELU → LayerScale → Residual"""
    def __init__(self, dim, layer_scale_init_value=1e-6):
        super().__init__()
        self.dwconv = nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim)
        self.norm = LayerNorm2d(dim)
        self.pwconv1 = nn.Conv2d(dim, 4 * dim, kernel_size=1)
        self.act = nn.GELU()
        self.pwconv2 = nn.Conv2d(4 * dim, dim, kernel_size=1)
        self.gamma = nn.Parameter(layer_scale_init_value * torch.ones(dim)) if layer_scale_init_value > 0 else None

    def forward(self, x):
        shortcut = x
        x = self.dwconv(x)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.pwconv2(x)
        if self.gamma is not None:
            x = self.gamma.view(1, -1, 1, 1) * x
        return shortcut + x


class ConvNeXtTiny(nn.Module):
    """
    ConvNeXt-Tiny 변형 (128x128 입력)
    구조: Stem(4x4,s4) → [Block×{3,3,9,3} + Downsample] → GAP → Dropout → FC
    """
    def __init__(self, num_classes=2):
        super().__init__()
        depths = [3, 3, 9, 3]
        dims = [96, 192, 384, 768]

        self.downsample_layers = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(3, dims[0], kernel_size=4, stride=4),  # 128 → 32
                LayerNorm2d(dims[0])
            )
        ])
        for i in range(3):
            self.downsample_layers.append(
                nn.Sequential(
                    LayerNorm2d(dims[i]),
                    nn.Conv2d(dims[i], dims[i+1], kernel_size=2, stride=2)
                )
            )

        self.stages = nn.ModuleList()
        for i in range(4):
            blocks = [ConvNeXtBlock(dims[i]) for _ in range(depths[i])]
            self.stages.append(nn.ModuleList(blocks))

        self.norm = LayerNorm2d(dims[-1])
        self.head = nn.Linear(dims[-1], num_classes)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.downsample_layers[0](x)
        for i in range(4):
            for block in self.stages[i]:
                x = block(x)
            if i < 3:
                x = self.downsample_layers[i+1](x)
        x = self.norm(x)
        x = x.mean([-2, -1])
        x = self.dropout(x)
        return self.head(x)

print("ConvNeXt-Tiny: Stem(4x4,s4) → Blocks×{3,3,9,3} → GAP → D → FC")
print(ConvNeXtTiny())
print(f"파라미터: {sum(p.numel() for p in ConvNeXtTiny().parameters()):,}")



## 5. 학습 함수

In [ ]:
# --------------------------------------------
# 학습/평가 유틸리티 함수 (H100 최적화)
# --------------------------------------------
scaler = GradScaler('cuda', enabled=USE_AMP)

def train_epoch(model, loader, criterion, optimizer):
    """Mixed Precision 학습 (H100 최적화)"""
    model.train()
    for images, labels in loader:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        
        optimizer.zero_grad(set_to_none=True)  # 메모리 효율적
        
        with autocast('cuda', enabled=USE_AMP):
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

def evaluate(model, loader):
    """평가 함수 - Accuracy, F1 (Micro), F1 (Macro) 반환"""
    model.eval()
    preds, labels_list = [], []
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device, non_blocking=True)
            with autocast('cuda', enabled=USE_AMP):
                outputs = model(images)
            preds.extend(outputs.argmax(1).cpu().numpy())
            labels_list.extend(labels.numpy())
    
    acc = accuracy_score(labels_list, preds)
    f1_micro = f1_score(labels_list, preds, average='micro')
    f1_macro = f1_score(labels_list, preds, average='macro')
    return acc, f1_micro, f1_macro

def train_model(model, train_loader, val_loader, epochs=20, lr=0.001):
    """H100 최적화 학습 함수"""
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    best_acc = 0
    best_state = None
    patience = 0
    
    for epoch in range(epochs):
        train_epoch(model, train_loader, criterion, optimizer)
        val_acc, _, _ = evaluate(model, val_loader)
        scheduler.step()
        
        if val_acc > best_acc:
            best_acc = val_acc
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            patience = 0
        else:
            patience += 1
            if patience >= 7:  # H100에서는 여유 있게
                break
    
    if best_state:
        model.load_state_dict(best_state)
    return model

print("H100 최적화 학습 함수 정의 완료")
print("  - Mixed Precision (autocast + GradScaler)")
print("  - F1 Score: Micro (=Accuracy) + Macro (클래스별 평균)")



## 6. RepeatedStratifiedKFold 5겹 × 5회 교차검증

### 검증 방법
- **RepeatedStratifiedKFold**: sklearn에서 제공하는 반복 계층화 교차검증
- **n_splits=5**: 5겹 교차검증 (데이터를 5개 폴드로 분할)
- **n_repeats=5**: 5회 반복 (서로 다른 무작위 시드로 분할)
- **총 학습 횟수**: 5 × 5 = 25회

### 장점
1. 단순 5겹 교차검증보다 더 안정적인 성능 추정
2. 서로 다른 무작위 분할로 분할 편향 감소
3. 표준편차 계산에 더 많은 샘플 (25개 vs 5개)

In [ ]:
# --------------------------------------------
# 인덱스 기반 부분집합을 transform과 함께 래핑하는 헬퍼
# --------------------------------------------
class TransformDataset(Dataset):
    def __init__(self, base_dataset, indices, transform):
        self.base = base_dataset
        self.indices = indices
        self.transform = transform
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        i = self.indices[idx]
        image = Image.open(self.base.image_paths[i]).convert('RGB')
        return self.transform(image), self.base.labels[i]



In [ ]:
# --------------------------------------------
# RepeatedStratifiedKFold 5겹 × 5회 교차검증 실행 함수
# - n_splits=5, n_repeats=5 → 총 25회 학습
# - 각 반복마다 train/val 분리 후 테스트 세트는 고정 평가
# - torch.compile로 추가 최적화 시도
# --------------------------------------------
def run_repeated_kfold(model_class, model_name, n_splits=5, n_repeats=5):
    """
    H100 최적화 RepeatedStratifiedKFold
    
    Args:
        model_class: 모델 클래스
        model_name: 모델 이름 (출력용)
        n_splits: 교차검증 폴드 수 (기본 5)
        n_repeats: 반복 횟수 (기본 5)
    
    Returns:
        results: 25회 테스트 성능 딕셔너리
    """
    results = {
        'test_acc': [], 'test_f1_micro': [], 'test_f1_macro': []
    }
    
    print()
    print('='*60)
    print(f'{model_name}')
    print(f'RepeatedStratifiedKFold: {n_splits}겹 × {n_repeats}회 = {n_splits * n_repeats}회 학습')
    print('='*60)
    
    # RepeatedStratifiedKFold 설정
    rskf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)
    all_indices = np.arange(len(train_dataset))
    
    # 테스트 로더 (고정)
    test_loader = DataLoader(
        test_dataset, 
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY
    )
    
    # 총 반복 횟수
    total_iterations = n_splits * n_repeats
    
    for i, (train_idx, val_idx) in enumerate(rskf.split(all_indices, train_dataset.labels)):
        repeat_num = i // n_splits + 1  # 현재 반복 번호 (1~5)
        fold_num = i % n_splits + 1     # 현재 폴드 번호 (1~5)
        
        set_seed(42 + i)
        
        # 진행 상황 출력
        print(f"[{i+1:2d}/{total_iterations}] 반복{repeat_num}-폴드{fold_num} | Train: {len(train_idx)}, Val: {len(val_idx)}", end=" → ")
        
        train_subset = TransformDataset(train_dataset, train_idx, train_transform)
        val_subset = TransformDataset(train_dataset, val_idx, test_transform)
        
        train_loader = DataLoader(
            train_subset, 
            batch_size=BATCH_SIZE, 
            shuffle=True,
            num_workers=NUM_WORKERS,
            pin_memory=PIN_MEMORY,
            drop_last=True
        )
        val_loader = DataLoader(
            val_subset, 
            batch_size=BATCH_SIZE,
            num_workers=NUM_WORKERS,
            pin_memory=PIN_MEMORY
        )
        
        # 모델 초기화
        model = model_class().to(device)
        
        # PyTorch 2.x: torch.compile 최적화
        if hasattr(torch, 'compile'):
            try:
                model = torch.compile(model, mode='reduce-overhead')
            except:
                pass
        
        # 학습
        model = train_model(model, train_loader, val_loader, epochs=30)
        
        # 테스트 평가
        test_acc, test_f1_micro, test_f1_macro = evaluate(model, test_loader)
        
        results['test_acc'].append(test_acc)
        results['test_f1_micro'].append(test_f1_micro)
        results['test_f1_macro'].append(test_f1_macro)
        
        print(f"Test Acc: {test_acc:.4f}, F1(Micro): {test_f1_micro:.4f}, F1(Macro): {test_f1_macro:.4f}")
        
        # 메모리 정리
        del model
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
    
    # 최종 요약
    print()
    print(f"{'='*60}")
    print(f"[{model_name}] 최종 결과 (25회 평균)")
    print(f"  Accuracy:   {np.mean(results['test_acc']):.4f} ± {np.std(results['test_acc']):.4f}")
    print(f"  F1 (Micro): {np.mean(results['test_f1_micro']):.4f} ± {np.std(results['test_f1_micro']):.4f}")
    print(f"  F1 (Macro): {np.mean(results['test_f1_macro']):.4f} ± {np.std(results['test_f1_macro']):.4f}")
    
    return results

In [ ]:
lenet_results = run_repeated_kfold(LeNet5, "LeNet-5 (베이스라인)")

In [ ]:
vgg_results = run_repeated_kfold(VGGNet, "VGGNet-style")

In [ ]:
lite_results = run_repeated_kfold(EfficientNetLite, "EfficientNet-Lite")

In [ ]:
resnet_results = run_repeated_kfold(ResNetCNN, "ResNetCNN")

In [ ]:
b0_results = run_repeated_kfold(EfficientNetB0, "EfficientNet-B0 (최고 성능)")

In [ ]:
convnext_results = run_repeated_kfold(ConvNeXtTiny, "ConvNeXt-Tiny (고성능)")

## 7. 결과 분석

In [ ]:
# --------------------------------------------
# 결과 요약 함수 (RepeatedStratifiedKFold용)
# - 25회 결과를 5회 반복 × 5겹 형태로 정리
# --------------------------------------------
def print_repeated_kfold_summary(results, name, n_splits=5, n_repeats=5):
    """
    RepeatedStratifiedKFold 결과를 반복별/폴드별로 정리하여 출력
    """
    print()
    print(f"{'='*80}")
    print(f"{name} - RepeatedStratifiedKFold 결과 ({n_splits}겹 × {n_repeats}회)")
    print(f"{'='*80}")
    
    # 반복별 평균 계산
    print(f"\n{'반복별 평균 성능':^40}")
    print("-"*70)
    print(f"{'반복':<8} {'Accuracy':<18} {'F1(Micro)':<18} {'F1(Macro)':<18}")
    print("-"*70)
    
    for r in range(n_repeats):
        start_idx = r * n_splits
        end_idx = start_idx + n_splits
        
        acc_mean = np.mean(results['test_acc'][start_idx:end_idx])
        f1mi_mean = np.mean(results['test_f1_micro'][start_idx:end_idx])
        f1ma_mean = np.mean(results['test_f1_macro'][start_idx:end_idx])
        
        print(f"반복 {r+1:<4} {acc_mean:.4f}             {f1mi_mean:.4f}             {f1ma_mean:.4f}")
    
    print("-"*70)
    
    # 전체 평균 및 표준편차
    acc_mean, acc_std = np.mean(results['test_acc']), np.std(results['test_acc'])
    f1_micro_mean, f1_micro_std = np.mean(results['test_f1_micro']), np.std(results['test_f1_micro'])
    f1_macro_mean, f1_macro_std = np.mean(results['test_f1_macro']), np.std(results['test_f1_macro'])
    
    print(f"\n전체 (25회)")
    print(f"  Accuracy:   {acc_mean:.4f} ± {acc_std:.4f}")
    print(f"  F1 (Micro): {f1_micro_mean:.4f} ± {f1_micro_std:.4f}")
    print(f"  F1 (Macro): {f1_macro_mean:.4f} ± {f1_macro_std:.4f}")
    
    return acc_mean, acc_std, f1_micro_mean, f1_micro_std, f1_macro_mean, f1_macro_std

print("="*60)
print("Final Results (RepeatedStratifiedKFold: 5겹 × 5회 = 25회)")
print("="*60)
lenet_stats = print_repeated_kfold_summary(lenet_results, "LeNet-5 (Baseline)")
vgg_stats = print_repeated_kfold_summary(vgg_results, "VGGNet-style")
resnet_stats = print_repeated_kfold_summary(resnet_results, "ResNetCNN")
lite_stats = print_repeated_kfold_summary(lite_results, "EfficientNet-Lite")
b0_stats = print_repeated_kfold_summary(b0_results, "EfficientNet-B0 (Best)")
convnext_stats = print_repeated_kfold_summary(convnext_results, "ConvNeXt-Tiny (Best)")

In [ ]:
# --------------------------------------------
# 시각화 - RepeatedStratifiedKFold 결과 (5겹 × 5회 = 25회)
# --------------------------------------------

# 한글 폰트 재설정
import platform
if platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = 'AppleGothic'
elif platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
else:  # Linux
    font_list = [f.name for f in fm.fontManager.ttflist if 'Nanum' in f.name or 'Gothic' in f.name]
    if font_list:
        plt.rcParams['font.family'] = font_list[0]
plt.rcParams['axes.unicode_minus'] = False

# 모델별 결과 정리
models = ['LeNet-5', 'VGGNet', 'ResNetCNN', 'Eff-Lite', 'Eff-B0', 'ConvNeXt']
all_results = [lenet_results, vgg_results, resnet_results, lite_results, b0_results, convnext_results]
colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99', '#ff99ff', '#c2a3ff']

# ============================================
# 그래프 1: 모든 평가 지표 비교 (25회 평균 ± 표준편차)
# ============================================
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Accuracy
acc_means = [np.mean(r['test_acc']) for r in all_results]
acc_stds = [np.std(r['test_acc']) for r in all_results]
bars1 = axes[0].bar(models, acc_means, yerr=acc_stds, capsize=5, color=colors, edgecolor='black', linewidth=0.5)
axes[0].set_title('Accuracy (정확도)', fontsize=12, fontweight='bold')
axes[0].set_ylim([0.5, 1.0])
axes[0].set_ylabel('Score')
axes[0].tick_params(axis='x', rotation=15)
for bar, mean, std in zip(bars1, acc_means, acc_stds):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 0.01, 
                 f'{mean:.3f}', ha='center', va='bottom', fontsize=8)

# 2. F1 Score (Micro)
f1_micro_means = [np.mean(r['test_f1_micro']) for r in all_results]
f1_micro_stds = [np.std(r['test_f1_micro']) for r in all_results]
bars2 = axes[1].bar(models, f1_micro_means, yerr=f1_micro_stds, capsize=5, color=colors, edgecolor='black', linewidth=0.5)
axes[1].set_title('F1 Score (Micro)', fontsize=12, fontweight='bold')
axes[1].set_ylim([0.5, 1.0])
axes[1].set_ylabel('Score')
axes[1].tick_params(axis='x', rotation=15)
for bar, mean, std in zip(bars2, f1_micro_means, f1_micro_stds):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 0.01, 
                 f'{mean:.3f}', ha='center', va='bottom', fontsize=8)

# 3. F1 Score (Macro)
f1_macro_means = [np.mean(r['test_f1_macro']) for r in all_results]
f1_macro_stds = [np.std(r['test_f1_macro']) for r in all_results]
bars3 = axes[2].bar(models, f1_macro_means, yerr=f1_macro_stds, capsize=5, color=colors, edgecolor='black', linewidth=0.5)
axes[2].set_title('F1 Score (Macro)', fontsize=12, fontweight='bold')
axes[2].set_ylim([0.5, 1.0])
axes[2].set_ylabel('Score')
axes[2].tick_params(axis='x', rotation=15)
for bar, mean, std in zip(bars3, f1_macro_means, f1_macro_stds):
    axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 0.01, 
                 f'{mean:.3f}', ha='center', va='bottom', fontsize=8)

plt.suptitle('Chihuahua vs Muffin - 모델별 성능 비교\n(RepeatedStratifiedKFold: 5겹 × 5회 = 25회)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('task2_metrics_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

# ============================================
# 그래프 2: 세 지표를 하나의 그래프에 그룹으로 표시
# ============================================
fig, ax = plt.subplots(figsize=(16, 6))

x = np.arange(len(models))
width = 0.25

bars_acc = ax.bar(x - width, acc_means, width, yerr=acc_stds, label='Accuracy', color='#4ECDC4', capsize=3, edgecolor='black', linewidth=0.5)
bars_micro = ax.bar(x, f1_micro_means, width, yerr=f1_micro_stds, label='F1 (Micro)', color='#FF6B6B', capsize=3, edgecolor='black', linewidth=0.5)
bars_macro = ax.bar(x + width, f1_macro_means, width, yerr=f1_macro_stds, label='F1 (Macro)', color='#45B7D1', capsize=3, edgecolor='black', linewidth=0.5)

ax.set_xlabel('모델', fontsize=12)
ax.set_ylabel('Score', fontsize=12)
ax.set_title('Chihuahua vs Muffin - 평가 지표별 모델 성능 비교 (25회 평균)', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(models, fontsize=10)
ax.legend(loc='lower right', fontsize=10)
ax.set_ylim([0.5, 1.0])
ax.grid(axis='y', alpha=0.3)

for bar, mean in zip(bars_acc, acc_means):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
            f'{mean:.3f}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.savefig('task2_grouped_metrics.png', dpi=150, bbox_inches='tight')
plt.show()

# ============================================
# 그래프 3: 반복별 F1 (Macro) 성능 비교 (5회 반복 평균)
# ============================================
fig, ax = plt.subplots(figsize=(14, 5))

n_splits = 5
n_repeats = 5
x = np.arange(n_repeats)
width = 0.13

for i, (result, model, color) in enumerate(zip(all_results, models, colors)):
    # 각 반복별 평균 계산
    repeat_means = []
    for r in range(n_repeats):
        start_idx = r * n_splits
        end_idx = start_idx + n_splits
        repeat_means.append(np.mean(result['test_f1_macro'][start_idx:end_idx]))
    
    offset = (i - 2.5) * width
    ax.bar(x + offset, repeat_means, width, label=model, color=color, edgecolor='black', linewidth=0.5)

ax.set_xlabel('반복 횟수 (Repeat)', fontsize=12)
ax.set_ylabel('F1 Score (Macro)', fontsize=12)
ax.set_title('Chihuahua vs Muffin - 반복별 F1 (Macro) 성능 (각 반복은 5겹 평균)', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels([f'반복 {i+1}' for i in x], fontsize=10)
ax.legend(loc='lower right', fontsize=9, ncol=2)
ax.set_ylim([0.5, 1.0])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('task2_repeat_f1macro.png', dpi=150, bbox_inches='tight')
plt.show()

# ============================================
# 그래프 4: 박스플롯 - 25회 결과 분포
# ============================================
fig, ax = plt.subplots(figsize=(14, 6))

# 박스플롯 데이터 준비
box_data = [r['test_acc'] for r in all_results]
positions = np.arange(len(models))

bp = ax.boxplot(box_data, positions=positions, widths=0.6, patch_artist=True)

for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax.set_xticks(positions)
ax.set_xticklabels(models, fontsize=10)
ax.set_ylabel('Accuracy', fontsize=12)
ax.set_title('Chihuahua vs Muffin - Accuracy 분포 (25회 결과)', fontsize=14, fontweight='bold')
ax.set_ylim([0.5, 1.0])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('task2_boxplot_accuracy.png', dpi=150, bbox_inches='tight')
plt.show()

# ============================================
# 결과 요약 테이블 출력
# ============================================
print("\n" + "="*90)
print("📊 Chihuahua vs Muffin - 최종 결과 요약")
print("   (RepeatedStratifiedKFold: 5겹 × 5회 = 25회)")
print("="*90)
print(f"{'모델':<18} {'Accuracy':<20} {'F1 (Micro)':<20} {'F1 (Macro)':<20}")
print("-"*90)
for model, result in zip(models, all_results):
    acc_m, acc_s = np.mean(result['test_acc']), np.std(result['test_acc'])
    f1mi_m, f1mi_s = np.mean(result['test_f1_micro']), np.std(result['test_f1_micro'])
    f1ma_m, f1ma_s = np.mean(result['test_f1_macro']), np.std(result['test_f1_macro'])
    print(f"{model:<18} {acc_m:.4f} ± {acc_s:.4f}      {f1mi_m:.4f} ± {f1mi_s:.4f}      {f1ma_m:.4f} ± {f1ma_s:.4f}")
print("="*90)
print("\n※ 참고: 균형 데이터셋에서 F1 (Micro) = Accuracy 입니다.")
print("※ F1 (Macro)는 각 클래스(치와와/머핀)의 F1을 평균하여 클래스별 성능을 균등하게 반영합니다.")
print("※ RepeatedStratifiedKFold는 5겹 교차검증을 5회 반복하여 총 25회 학습합니다.")

# ============================================
# 파라미터 수 비교
# ============================================
print("\n" + "="*60)
print("📐 모델별 파라미터 수")
print("="*60)
models_params = [
    ("LeNet-5", LeNet5()),
    ("VGGNet", VGGNet()),
    ("ResNetCNN", ResNetCNN()),
    ("EfficientNet-Lite", EfficientNetLite()),
    ("EfficientNet-B0", EfficientNetB0()),
    ("ConvNeXt-Tiny", ConvNeXtTiny())
]
for name, model in models_params:
    params = sum(p.numel() for p in model.parameters())
    print(f"{name:20s}: {params:>12,} params")

## 8. 결론

### 검증 방법: RepeatedStratifiedKFold

```python
from sklearn.model_selection import RepeatedStratifiedKFold

rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
# 총 25회 학습 (5겹 × 5회 반복)
```

**장점**:
- 단순 5겹 교차검증보다 더 안정적인 성능 추정
- 서로 다른 무작위 분할로 분할 편향 감소
- 표준편차 계산에 더 많은 샘플 (25개 vs 5개)

---

### 모델별 성능 비교

| 모델 | 연도 | 핵심 구조 | 파라미터 | 예상 성능 |
|------|------|----------|---------|----------|
| LeNet-5 | 1998 | C(5×5)→Tanh→AvgP | ~560K | 낮음 (베이스라인) |
| VGGNet | 2014 | [C(3×3)→BN→R]×n→MaxP | ~1.9M | 중간 |
| ResNetCNN | 2015 | Residual Block + Skip | ~600K | 중상 |
| EfficientNet-Lite | 2019 | MBConv(k3) + SE | ~1.2M | 높음 |
| EfficientNet-B0 | 2019 | MBConv(k3,k5) + SE | ~4.0M | 최고 |
| ConvNeXt-Tiny | 2022 | DWConv(7×7) + LN + GELU | ~28M | 최고 |

※ 실제 성능은 위 시각화 결과 참조

---

### 평가 척도 해석

| 척도 | 계산 방식 | 의미 |
|------|----------|------|
| **Accuracy** | (TP+TN) / 전체 | 전체 정확도 |
| **F1 (Micro)** | 전체 TP,FP,FN 합산 | 균형 데이터에서 Accuracy와 동일 |
| **F1 (Macro)** | 클래스별 F1 평균 | 치와와/머핀 각각의 성능 균등 반영 |

※ 균형 데이터셋에서 F1 (Micro) = Accuracy는 sklearn의 정상 동작입니다.

---

### 모델별 상세 분석

#### 1. LeNet-5 (베이스라인, 1998)
- 수업시간에 배운 고전적 CNN 구조
- 5×5 커널과 Tanh 활성화 사용
- **한계**: 얕은 구조로 치와와 눈과 머핀 초콜릿칩의 세밀한 차이 구분 어려움

#### 2. VGGNet-style (2014)
- 작은 3×3 커널을 깊게 쌓아 수용 영역 확보
- BN + ReLU로 학습 안정화
- **장점**: LeNet-5보다 더 세밀한 특징 추출 가능
- **한계**: Skip connection 없어 깊은 학습 시 gradient 문제

#### 3. ResNetCNN (2015)
- Residual Connection으로 gradient flow 개선
- H(x) = F(x) + x로 잔차 학습
- **장점**: 더 깊은 네트워크 학습 가능, 세밀한 특징 보존

#### 4. EfficientNet-Lite
- 간소화된 MBConv 구조 (3×3 커널만 사용)
- SE Block으로 채널 attention
- **장점**: 원본 B0보다 가볍고 빠른 학습
- **한계**: 5×5 커널 미사용으로 수용 영역 제한

#### 5. EfficientNet-B0 (2019)
- 원본 논문의 검증된 아키텍처
- 3×3과 5×5 커널 혼합 사용 → 다양한 수용 영역
- MBConv6 (expand_ratio=6)로 충분한 표현력
- **장점**: ImageNet에서 검증된 구조, SE Block으로 채널 attention

#### 6. ConvNeXt-Tiny (2022) - 최신 아키텍처
- Vision Transformer의 설계 원칙을 순수 CNN에 적용
- **핵심 특징**:
  - Depthwise Conv 7×7: 가장 넓은 수용 영역
  - LayerNorm: 배치 크기에 독립적인 정규화
  - GELU: smooth한 활성화 함수
  - LayerScale: 안정적인 깊은 네트워크 학습
- **장점**: Transformer 수준 성능을 CNN으로 달성

---

### CNN 아키텍처 발전 흐름

```
LeNet-5 (1998)     VGGNet (2014)      ResNet (2015)      EfficientNet (2019)   ConvNeXt (2022)
     │                  │                  │                    │                   │
  5×5 커널         3×3 깊게 쌓기      Residual Conn.      MBConv + SE        ViT 원칙 적용
     │                  │                  │                    │                   │
  Tanh + AvgP       BN + ReLU          Skip + BN          Swish + SE        LN + GELU + 7×7
```

---

### 치와와 vs 머핀: 왜 어려운 분류 문제인가?

| 특징 | 치와와 | 머핀 | 구분 난이도 |
|------|--------|------|------------|
| 전체 형태 | 둥근 얼굴 | 둥근 윗면 | 매우 어려움 |
| 색상 | 갈색 털 | 갈색 반죽 | 매우 어려움 |
| 점 패턴 | 눈, 코 | 초콜릿칩 | 매우 어려움 |
| 질감 | 부드러운 털 | 거친 표면 | 중간 |
| 귀/가장자리 | 뾰족한 귀 | 없음 | 쉬움 |

### 핵심 인사이트

**치와와와 머핀을 구분하는 가장 중요한 특징 = 귀 형태**

- 치와와: 뾰족한 귀가 있음
- 머핀: 귀 형태가 없음

→ **넓은 수용 영역**을 가진 모델이 귀 형태를 인식하는 데 유리
- EfficientNet-B0: 5×5 커널로 넓은 수용 영역
- ConvNeXt-Tiny: 7×7 Depthwise로 가장 넓은 수용 영역

---

### 결론

**EfficientNet-B0와 ConvNeXt-Tiny**가 가장 높은 성능을 보일 것으로 예상:

1. **EfficientNet-B0**: 
   - 5×5 커널로 넓은 수용 영역
   - SE Block으로 채널 attention
   - ImageNet에서 검증된 구조

2. **ConvNeXt-Tiny**: 
   - 7×7 Depthwise로 가장 넓은 수용 영역
   - 최신 설계 원칙 (LayerNorm, GELU, LayerScale)
   - Transformer 수준의 성능을 CNN으로 달성

이 문제에서는 **넓은 수용 영역 + 채널 Attention**이 성능의 핵심입니다.

---

### RepeatedStratifiedKFold의 이점

| 항목 | 단순 5겹 CV | RepeatedStratifiedKFold (5×5) |
|------|------------|------------------------------|
| 총 학습 횟수 | 5회 | 25회 |
| 표준편차 샘플 수 | 5개 | 25개 |
| 분할 편향 | 한 번의 분할에 의존 | 5번의 다른 분할로 평균 |
| 결과 안정성 | 낮음 | 높음 |
| 계산 비용 | 낮음 | 5배 증가 |

→ 더 신뢰할 수 있는 성능 추정을 위해 RepeatedStratifiedKFold 사용